<a href="https://colab.research.google.com/github/derkmed/COMSE6998_025_Project/blob/main/pydrive_get_video_file_player_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab allows a user to extract all video files in a specified Google Drive.
The videos are returned as video player links for easy viewing.

In [25]:
# This should be the Public Google Drive Folder ID that contains all workout videos.
GOOGLE_DRIVE_WORKOUT_ID = '1U7FYKeoD669CtNtN64PBI4-tPkTlW05p' #@param {type: 'string'}

In [33]:
from google.auth import default
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import random
from typing import *

In [27]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
workout_folder_ids = set()
for file in drive.ListFile({'q': f"'{GOOGLE_DRIVE_WORKOUT_ID}' in parents",}).GetList():
    print('title {}, id {}'.format(file['title'], file['id']))
    workout_folder_ids.add(file['id'])


title russian twist, id 1qOHaHj5S1mbbSlFkPgGAXo1LBS4zcxSx
title romanian deadlift, id 1zVlLga5ACkicT6LglbzkmlgdSvs0mDE5
title hip thrust, id 1RxdAN-9ydzPpT02g53GSVX8xHwMdm1hn
title shoulder press, id 1lLWuVjAYiZyzmXy3gq75NwE4URl91wgs
title pull Up, id 1bfQluf50neeBei_JvZL64Zq69d2o0r7g
title plank, id 1LkbIvJvlQZliRPpPIdKnT4_RYl7Z1bJA
title bench press, id 13SbUgXnglw6syYywfnI-kwuXE3vqx82O
title decline bench press, id 1x_Q6br5hrslDXO4TCbooo313-UixcUxp
title leg raises, id 1qtduCup5jyayWO3zWwGGJv0YSbJkAyf9
title lateral raise, id 1Z6MiSUsCr7ZW4QmIb8LLMzTMABk1eBLZ
title leg extension, id 10SwcRU9HkzAZZuy-usKUmPQYdu_zDFr4
title t bar row, id 1Bww-S0NTE2FYrhi0lrf-dUozFxN7Rzqu
title incline bench press, id 1yh2V89BPzg735DcV7r-LIhxPwwIlQigJ
title deadlift, id 11tF2CPVp_z_20ifsiiE9_oCjzcwLuk99
title chest fly machine, id 1bYtOce16YFf1o4dGgeMClNsFJeDtbHtZ
title tricep Pushdown, id 1ScY9hAOaFYGBjPSGzyQwFY34_f4v5VLW
title barbell biceps curl, id 1msW4xUpSvubKWZnajnJuxQv0ix8hAISE
title squat, id 

In [43]:
def get_video_link(id: str) -> str:
  '''
  Parses the Google Drive Link given a folder ID.
  '''
  return f"https://drive.google.com/file/d/{id}/view?usp=drive_link"


def get_video_links(drive_folder_ids: Set[str]) -> Dict[str, str]:
  '''
  Returns a dictionary of files contained within a set of Drive folders mapped to their
  video player IDs. ASSUMES that all contained files are of video formats.
  '''
  video_links = {
      'video_title': [],
      'video_link': [],
      }
  for id in drive_folder_ids:
    for file in drive.ListFile({'q': f"'{id}' in parents",}).GetList():
      video_title, video_link =  file['title'], get_video_link(file['id'])
      video_links['video_title'].append(video_title)
      video_links['video_link'].append(video_link)
  return video_links


video_links = get_video_links(workout_folder_ids)
video_links_df = pd.DataFrame.from_dict(video_links)
print(f"There are {len(video_links_df)} videos in total")

# Used to randomly derive the instruction family examples.
# inst_family_examples_df = video_links_df[['video_title', 'video_link']].sample(21)
# gsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1cLqHKOCwNuqJ_e2lH0cdKnI8oi-u3lTsPHdL0dGRcyk/edit#gid=1916776205")
# inst_family_examples_sheet = gsheet.worksheet('Sheet8')
# set_with_dataframe(inst_family_examples_sheet, inst_family_examples_df)

There are 653 videos in total


In [65]:
#@title Set up Sheets Authentication
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [46]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1DfrjBxWfS2xBPC_MLYKBa29BNIbPy5LeFmNGzzO7i38/edit#gid=263001316" #@param {type: "string"}
INSTRUCTION_FAMILY_SHEET_NAME = "Instruction_Families" #@param (type: "string")
RD1_SHEET_NAME = "round1_annotation" #@param (type: "string")

gsheet = gc.open_by_url(SHEET_URL)
inst_families_sheet_data = gsheet.worksheet(INSTRUCTION_FAMILY_SHEET_NAME).get_all_values()
rd1_sheet_data = gsheet.worksheet(RD1_SHEET_NAME).get_all_values()
inst_fam_df = pd.DataFrame(inst_families_sheet_data[1:], columns=inst_families_sheet_data[0])
print(f"There are {len(inst_fam_df)} instruction families")

There are 21 instruction families


In [48]:
inst_fam_df['tmp'] = 0
video_links_df['tmp'] = 0

crossed_df = video_links_df.merge(inst_fam_df, on='tmp', how='outer')
print(f"There are {len(crossed_df)} parings")

There are 13713 parings


In [67]:
crossed_df.drop(columns='tmp')

,video_title,video_link,Instruction Families,About
0,lateral raise_7.mp4,https://drive.google.com/file/d/1QQ71WMbMQJPJ3...,Alternatives,Write a question asking about similar or relat...
1,lateral raise_7.mp4,https://drive.google.com/file/d/1QQ71WMbMQJPJ3...,Accesibility,Write a question regarding the accessibility o...
2,lateral raise_7.mp4,https://drive.google.com/file/d/1QQ71WMbMQJPJ3...,Anatomy,Write a question related to human anatomy
3,lateral raise_7.mp4,https://drive.google.com/file/d/1QQ71WMbMQJPJ3...,Answer-Eliciting,Write a question that elicits some sort of answer
4,lateral raise_7.mp4,https://drive.google.com/file/d/1QQ71WMbMQJPJ3...,Background,"Write a question about the background, setting..."
...,...,...,...,...
13708,tricep pushdown_3.mp4,https://drive.google.com/file/d/1LUK9kTYQ1qmBT...,Mental Health,Write a question about the mental health benef...
13709,tricep pushdown_3.mp4,https://drive.google.com/file/d/1LUK9kTYQ1qmBT...,Outside Knowledge,Write a question that pertains to the video bu...
13710,tricep pushdown_3.mp4,https://drive.google.com/file/d/1LUK9kTYQ1qmBT...,Performance,Write a question regarding the performance asp...
13711,tricep pushdown_3.mp4,https://drive.google.com/file/d/1LUK9kTYQ1qmBT...,Sports,Write a question about sports that may benefit...


In [69]:
inst_family_examples_sheet = gsheet.worksheet('interim_round1_sheet')
set_with_dataframe(inst_family_examples_sheet, crossed_df)